<a href="https://colab.research.google.com/github/kumarshovit/TestRepo/blob/master/CropDetectionUsingCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import os 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/Agriculturedata/Priyanka/Crop_details (1).csv')
df.head()

,Unnamed: 0,path,crop,croplabel
0,0,/kaggle/input/kag2/sugarcane/sugarcane037ahs.jpeg,sugarcane,3
1,1,/kaggle/input/kag2/sugarcane/sugarcane034a.jpeg,sugarcane,3
2,2,/kaggle/input/kag2/sugarcane/sugarcane032arot....,sugarcane,3
3,3,/kaggle/input/kag2/sugarcane/sugarcane036ahs.jpeg,sugarcane,3
4,4,/kaggle/input/kag2/sugarcane/sugarcane023ahs.jpeg,sugarcane,3


In [ ]:
categories = os.listdir(r'/content/drive/MyDrive/Agriculturedata/Priyanka/kag2')
categories


['rice', 'maize', 'sugarcane', 'wheat', 'jute']

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPool2D
import cv2

In [ ]:
getData = ImageDataGenerator(rescale=1/255,
                             shear_range=0.3,
                             horizontal_flip=True,
                            vertical_flip= True,
                            rotation_range=90)

In [ ]:
kag2_data = getData.flow_from_directory(r'/content/drive/MyDrive/Agriculturedata/Priyanka/kag2',
                                       target_size=(224,224),
                                       class_mode='categorical',
                                       shuffle = True,
                                       batch_size=32)

Found 809 images belonging to 5 classes.


In [ ]:
vgg_model = VGG16(input_shape = (224,224,3),
                  include_top=False,
                  weights='imagenet',
                  pooling='avg')
for layers in vgg_model.layers:
    layers.trainable = False

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
model = Sequential()
model.add(vgg_model)
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(5,activation="softmax"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 5)                 645       
                                                                 
Total params: 15,043,653
Trainable params: 328,965
Non-t

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
num_trian_samples = 2409
batch_size = 400
epochs = 50

final_model = model.fit_generator(kag2_data,
                                 steps_per_epoch=num_trian_samples//batch_size,
                                 epochs=epochs)

<ipython-input-13-78bb0bfb3c5e>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  final_model = model.fit_generator(kag2_data,


Epoch 1/50
6/6 [==============================] - 140s 22s/step - loss: 1.8362 - accuracy: 0.1875
Epoch 2/50
6/6 [==============================] - 128s 20s/step - loss: 1.6617 - accuracy: 0.2135
Epoch 3/50
6/6 [==============================] - 127s 20s/step - loss: 1.5858 - accuracy: 0.2500
Epoch 4/50
6/6 [==============================] - 129s 21s/step - loss: 1.5675 - accuracy: 0.2292
Epoch 5/50
6/6 [==============================] - 126s 20s/step - loss: 1.5371 - accuracy: 0.3229
Epoch 6/50
6/6 [==============================] - 124s 20s/step - loss: 1.5547 - accuracy: 0.3073
Epoch 7/50
1/6 [====>.........................] - ETA: 1:57 - loss: 1.4656 - accuracy: 0.4375

KeyboardInterrupt: ignored

In [ ]:
model.save('crops_detection_model_50epochs.h5')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255)
crop_data = test_datagen.flow_from_directory('/content/drive/MyDrive/Agriculturedata/Priyanka/crop_images',
                                       target_size=(224,224),
                                       class_mode='categorical',
                                       shuffle = True,
                                       batch_size=32)

In [ ]:
plt.plot(final_model.history['loss'],label='train_loss')
plt.plot(final_model.history['accuracy'],label='train_acc')
plt.legend()
plt.show()

In [ ]:
class_labels = crop_data.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
Y_pred = model.predict_generator(crop_data)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
def predict_crop(path,actual,class_labels):
    img = cv2.imread(path)
    img = cv2.resize(img,(224,224))
    img = np.array(img).reshape((1,224,224,3))
    Y_pred = model.predict(img)
    y_pred = np.argmax(Y_pred,axis=1)
    if y_pred == actual:
        print("Correct Prediction 🎉🎊🎉")
    else:
        print("Wrong Prediction 💥💥💥")
    print('Actual class "{0}" and predicted class "{1}"'.format(class_labels[int(y_pred)],class_labels[actual]))

In [ ]:
predict_crop('/content/drive/MyDrive/Agriculturedata/Priyanka/test_crop_image/wheatss.jpg',4,class_labels)

In [ ]:
predict_crop('/content/drive/MyDrive/Agriculturedata/Priyanka/test_crop_image/maize000.jfif',1,class_labels)


In [ ]:
predict_crop('/content/drive/MyDrive/Agriculturedata/Priyanka/test_crop_image/jute03.jpg',0,class_labels)

In [ ]:
predict_crop('/content/drive/MyDrive/Agriculturedata/Priyanka/test_crop_image/wheat image.jpg',4,class_labels)